In [1]:
import numpy as np
from scipy import linalg, stats, ndimage
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.metrics import structural_similarity as ssim, peak_signal_noise_ratio as psnr
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import os
from utils.readNaImage import read_na_image

In [2]:
# Setup your data path for the 4D data

def get_4d_complex_data_provided(num_images=24, zero_imag=False):
    pat_dict_corrected = {}
    path = f'Das_Denoising_Data/24_images' # contains data from 6 * 4 runs
    dims = [64, 64, 64]
    
    for folder in os.listdir(path):
        if folder.startswith("Das_") or folder.startswith("H0"):
            common_path = os.path.join(path, folder)
            all_avgs_corrected = list()

            def process_images(path, dims):
                folder_name = path.split('/')[-2]
                print(folder_name)
                if not folder_name in ['H093_v2_tpiRecon', 'H094_v2_tpiRecon', 
                                     'H095_v2_tpiRecon', 'H096_v2_tpiRecon', 
                                     'H097_v2_tpiRecon', 'H098_v2_tpiRecon', 
                                     'H099_v2_tpiRecon']:
                    path = os.path.join(path, 'AVGall_kw1')

                try:
                    if os.path.exists(os.path.join(path, 'tpirec_image_PC2.c0')):
                        filename_corrected = 'tpirec_image_PC2.c0'
                    elif os.path.exists(os.path.join(path, 'tpirec_image_PC.c0')):
                        filename_corrected = 'tpirec_image_PC.c0'
                    elif os.path.exists(os.path.join(path, 'tpirec-_image_PC.c0')):
                        filename_corrected = 'tpirec-_image_PC.c0'
                    image3d_corrected = read_na_image(os.path.join(path, filename_corrected), 
                                                    *dims, 'complex')
                except:
                    print(f"Error reading {filename_corrected}")
                    image3d_corrected = np.zeros(tuple(dims) + (1,), dtype=np.complex64)

                return image3d_corrected

            run_count = 0
            for runs in os.listdir(common_path):
                if runs == '.DS_Store':
                    continue
                if run_count == num_images//6:
                    break
                image3d_corrected = process_images(os.path.join(common_path, runs), dims)
                all_avgs_corrected.append(image3d_corrected)
                run_count += 1

            if folder.startswith("Das_"):
                folder_key = folder.split("_")[1]
            else:
                folder_key = folder.split("_")[0]
            folder_key = folder_key.split("H0")[1]

            all_avgs_corrected = np.array(all_avgs_corrected)
            pat_dict_corrected[folder_key] = np.transpose(all_avgs_corrected, (1, 2, 3, 0))

    return pat_dict_corrected

In [6]:


def mp_threshold(S, M, N):
    """Calculate Marchenko-Pastur threshold for eigenvalues."""
    vals = S**2 / N
    scaling = np.ones(len(vals))
    if M > N:
        scaling = (M - np.arange(len(vals))) / N
        scaling[scaling < 1] = 1
    
    csum = np.cumsum(vals[::-1])[::-1]
    cmean = csum / np.arange(len(vals), 0, -1)
    sigmasq_1 = cmean / scaling
    
    gamma = (M - np.arange(len(vals))) / N
    rangeMP = 4 * np.sqrt(gamma)
    rangeData = vals - vals[-1]
    sigmasq_2 = rangeData / rangeMP
    
    t = np.argmax(sigmasq_2 < sigmasq_1)
    sigma = np.sqrt(sigmasq_1[t])
    
    return sigma, t

def improved_mppca(X, patch_radius=2, n_components=None):
    """Marchenko-Pastur PCA denoising with automatic component selection."""
    sx, sy, sz, N = X.shape
    patch_size = 2 * patch_radius + 1
    
    Xdn = np.zeros_like(X)
    weights = np.zeros((sx, sy, sz))
    sigma_map = np.zeros((sx, sy, sz))
    npars_map = np.zeros((sx, sy, sz))
    
    pad_width = ((patch_radius, patch_radius),
                 (patch_radius, patch_radius),
                 (patch_radius, patch_radius),
                 (0, 0))
    X_padded = np.pad(X, pad_width, mode='reflect')
    
    for x in tqdm(range(patch_radius, sx + patch_radius)):
        for y in range(patch_radius, sy + patch_radius):
            for z in range(patch_radius, sz + patch_radius):
                patch = X_padded[x-patch_radius:x+patch_radius+1,
                               y-patch_radius:y+patch_radius+1,
                               z-patch_radius:z+patch_radius+1]
                
                M = patch.reshape(-1, N)
                U, S, Vh = linalg.svd(M, full_matrices=False)
                
                if n_components is None:
                    sigma, npars = mp_threshold(S, M.shape[0], N)
                    S[npars:] = 0
                else:
                    npars = n_components
                    sigma = np.median(S) / 0.6745
                    S[n_components:] = 0
                
                M_denoised = U @ np.diag(S) @ Vh
                patch_denoised = M_denoised.reshape(patch_size, patch_size, patch_size, N)
                
                x_idx = x-patch_radius
                y_idx = y-patch_radius
                z_idx = z-patch_radius
                
                Xdn[x_idx, y_idx, z_idx] = patch_denoised[patch_radius, patch_radius, patch_radius]
                weights[x_idx, y_idx, z_idx] += 1
                sigma_map[x_idx, y_idx, z_idx] = sigma
                npars_map[x_idx, y_idx, z_idx] = npars
    
    Xdn = Xdn / weights[..., np.newaxis]
    return Xdn, np.mean(sigma_map), np.mean(npars_map)

def analyze_eigenvalues(X, patch_radius=2):
    """Analyze eigenvalue distribution for a sample of patches."""
    patch_size = 2 * patch_radius + 1
    sx, sy, sz, N = X.shape
    
    n_samples = 100
    eigenvalues = []
    cutoffs = []
    
    for _ in range(n_samples):
        x = np.random.randint(patch_radius, sx - patch_radius)
        y = np.random.randint(patch_radius, sy - patch_radius)
        z = np.random.randint(patch_radius, sz - patch_radius)
        
        patch = X[x-patch_radius:x+patch_radius+1,
                 y-patch_radius:y+patch_radius+1,
                 z-patch_radius:z+patch_radius+1]
        
        M = patch.reshape(-1, N)
        _, S, _ = linalg.svd(M, full_matrices=False)
        
        sigma, npars = mp_threshold(S, M.shape[0], N)
        
        eigenvalues.append(S**2 / N)
        cutoffs.append(npars)
        
    return np.array(eigenvalues), np.array(cutoffs)

def plot_all_subjects_results(results_dict, save_dir='analysis_plots'):
    """Create combined plots with confidence intervals and individual subject lines"""
    save_dir = Path(save_dir)
    save_dir.mkdir(exist_ok=True)

    # n_components_list = [None, 1, 2, 3, 4, 7]
    n_components_list = [None]
    # Collect metrics across subjects
    all_psnr = []
    all_sigma = []
    all_npars = []
    
    for subject_id, results in results_dict.items():
        all_psnr.append(results['psnr_scores'])
        all_sigma.append(results['sigma_list'])
        all_npars.append(results['npars_list'])
    
    # Convert to numpy arrays
    all_psnr = np.array(all_psnr)
    all_sigma = np.array(all_sigma)
    all_npars = np.array(all_npars)
    
    # Calculate statistics
    psnr_mean = np.mean(all_psnr, axis=0)
    psnr_std = np.std(all_psnr, axis=0)
    psnr_min = np.min(all_psnr, axis=0)
    psnr_max = np.max(all_psnr, axis=0)
    
    # Create aggregate plots
    plt.figure(figsize=(15, 10))
    
    # Individual subject lines
    for subject_id, results in results_dict.items():
        plt.plot(range(len(n_components_list)), results['psnr_scores'], 
                alpha=0.2, color='blue', linewidth=1)
    
    # Mean line with confidence band
    plt.plot(range(len(n_components_list)), psnr_mean, 'b-', 
            linewidth=2, label='Mean PSNR')
    plt.fill_between(range(len(n_components_list)), 
                    psnr_mean - psnr_std, 
                    psnr_mean + psnr_std, 
                    alpha=0.3, color='blue', 
                    label='±1 std dev')
    plt.fill_between(range(len(n_components_list)), 
                    psnr_min, psnr_max, 
                    alpha=0.1, color='blue', 
                    label='Min-Max range')
    
    plt.xticks(range(len(n_components_list)), 
               ['Auto' if x is None else str(x) for x in n_components_list])
    plt.xlabel('Number of Components')
    plt.ylabel('PSNR (dB)')
    plt.title('PSNR vs Components Across All Subjects')
    plt.grid(True)
    plt.legend()
    plt.savefig(save_dir / 'aggregate_psnr.png')
    plt.close()
    
    # Create detailed multi-panel figure
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 15))
    
    # 1. PSNR with confidence bands
    for subject_id, results in results_dict.items():
        ax1.plot(range(len(n_components_list)), results['psnr_scores'], 
                alpha=0.2, color='blue', linewidth=1)
    ax1.plot(range(len(n_components_list)), psnr_mean, 'b-', linewidth=2, label='Mean')
    ax1.fill_between(range(len(n_components_list)), 
                    psnr_mean - psnr_std, 
                    psnr_mean + psnr_std, 
                    alpha=0.3, color='blue')
    ax1.set_xticks(range(len(n_components_list)))
    ax1.set_xticklabels(['Auto' if x is None else str(x) for x in n_components_list])
    ax1.set_title('PSNR vs Components')
    ax1.set_xlabel('Number of Components')
    ax1.set_ylabel('PSNR (dB)')
    ax1.grid(True)
    
    # 2. Sigma values distribution
    sigma_mean = np.mean(all_sigma, axis=0)
    sigma_std = np.std(all_sigma, axis=0)
    for subject_id, results in results_dict.items():
        ax2.plot(range(len(n_components_list)), results['sigma_list'], 
                alpha=0.2, color='red', linewidth=1)
    ax2.plot(range(len(n_components_list)), sigma_mean, 'r-', linewidth=2)
    ax2.fill_between(range(len(n_components_list)), 
                    sigma_mean - sigma_std, 
                    sigma_mean + sigma_std, 
                    alpha=0.3, color='red')
    ax2.set_xticks(range(len(n_components_list)))
    ax2.set_xticklabels(['Auto' if x is None else str(x) for x in n_components_list])
    ax2.set_title('Noise Level (Sigma) vs Components')
    ax2.set_xlabel('Number of Components')
    ax2.set_ylabel('Sigma')
    ax2.grid(True)
    
    # 3. Box plot of PSNR distributions
    ax3.boxplot([all_psnr[:, i] for i in range(len(n_components_list))],
                labels=['Auto' if x is None else str(x) for x in n_components_list])
    ax3.set_title('PSNR Distribution by Components')
    ax3.set_xlabel('Number of Components')
    ax3.set_ylabel('PSNR (dB)')
    ax3.grid(True)
    
    # 4. Components used vs requested
    npars_mean = np.mean(all_npars, axis=0)
    npars_std = np.std(all_npars, axis=0)
    for subject_id, results in results_dict.items():
        ax4.plot(range(len(n_components_list)), results['npars_list'], 
                alpha=0.2, color='green', linewidth=1)
    ax4.plot(range(len(n_components_list)), npars_mean, 'g-', linewidth=2)
    ax4.fill_between(range(len(n_components_list)), 
                    npars_mean - npars_std, 
                    npars_mean + npars_std, 
                    alpha=0.3, color='green')
    ax4.set_xticks(range(len(n_components_list)))
    ax4.set_xticklabels(['Auto' if x is None else str(x) for x in n_components_list])
    ax4.set_title('Components Used vs Requested')
    ax4.set_xlabel('Requested Components')
    ax4.set_ylabel('Components Used')
    ax4.grid(True)
    
    plt.tight_layout()
    plt.savefig(save_dir / 'detailed_analysis.png')
    plt.close()


    # Save statistical summary
    summary = pd.DataFrame({
        'Components': n_components_list,
        'PSNR_mean': psnr_mean,
        'PSNR_std': psnr_std,
        'PSNR_min': psnr_min,
        'PSNR_max': psnr_max,
        'Sigma_mean': sigma_mean,
        'Sigma_std': sigma_std,
        'NPars_mean': npars_mean,
        'NPars_std': npars_std
    })
    summary.to_csv(save_dir / 'analysis_summary.csv')
    
    return summary

def comprehensive_analysis(data, patch_radius=2):
    """Analyze data with different numbers of components"""
    # n_components_list = [None, 1, 2, 3, 4, 7]
    n_components_list = [None]
    results = {
        'denoised_list': [],
        'sigma_list': [],
        'npars_list': [],
        'psnr_scores': [],
        'eigenvalues_list': [],
        'cutoffs_list': []
    }
    
    # Create brain mask
    middle_slice = np.abs(data[:, :, data.shape[2]//2, 0])
    threshold = np.mean(middle_slice) + 0.5 * np.std(middle_slice)
    mask = middle_slice > threshold
    
    # Reference for PSNR
    reference = data.real
    
    for n_comp in n_components_list:
        print(f"\nAnalyzing n_components = {n_comp}")
        
        # Run denoising
        denoised, sigma, npars = improved_mppca(data, 
                                              patch_radius=patch_radius,
                                              n_components=n_comp)
        
        # Analyze eigenvalues
        eigenvalues, cutoffs = analyze_eigenvalues(data, patch_radius=patch_radius)
        
        # Calculate PSNR
        slice_psnrs = []
        for z in range(data.shape[2]):
            ref_slice = reference[:, :, z, 0] * mask
            den_slice = denoised[:, :, z, 0].real * mask
            
            if np.any(mask):
                slice_psnr = psnr(ref_slice, den_slice, data_range=ref_slice.max())
                slice_psnrs.append(slice_psnr)
        
        avg_psnr = np.mean(slice_psnrs)
        
        # Store results
        results['denoised_list'].append(denoised)
        results['sigma_list'].append(sigma)
        results['npars_list'].append(npars)
        results['psnr_scores'].append(avg_psnr)
        results['eigenvalues_list'].append(eigenvalues)
        results['cutoffs_list'].append(cutoffs)
        
        print(f"Components: {n_comp if n_comp is not None else 'Auto'}")
        print(f"PSNR: {avg_psnr:.2f} dB")
        print(f"Sigma: {sigma:.2e}")
        print(f"Components used: {npars}")
    
    return results

# Main execution
if __name__ == "__main__":
    # Create output directory
    output_dir = Path('mppca_analysis_results')
    output_dir.mkdir(exist_ok=True)
    
    # Load data
    print("Loading data...")
    subject_dict = get_4d_complex_data_provided(24)
    
    # Process all subjects
    results_dict = {}
    
    for subject_id in subject_dict.keys():
        print(f"\nProcessing subject {subject_id}")
        data = subject_dict[subject_id]
        
        # Run analysis
        results = comprehensive_analysis(data, patch_radius=2)
        results_dict[subject_id] = results
        
        # Save subject-specific results
        np.save(output_dir / f'subject_{subject_id}_results.npy', results)
    
    # Create and save aggregate plots
    print("\nGenerating aggregate plots and statistics...")
    summary = plot_all_subjects_results(results_dict, save_dir=output_dir)
    
    print("\nAnalysis complete! Results saved in:", output_dir)
    print("\nSummary of results:")
    print(summary)

Loading data...
H077_v2_tpiRecon
H077_v2_tpiRecon
H077_v2_tpiRecon
Das_H049_v2
Das_H049_v2
Das_H049_v2
Das_H049_v2
H096_v2_tpiRecon
H096_v2_tpiRecon
H096_v2_tpiRecon
H083_v2_tpiRecon
H083_v2_tpiRecon
H083_v2_tpiRecon
H086_v2_tpiRecon
H086_v2_tpiRecon
H086_v2_tpiRecon
H072_v2_tpiRecon
H072_v2_tpiRecon
H072_v2_tpiRecon
Das_H048_v2
Das_H048_v2
Das_H048_v2
H093_v2_tpiRecon
H093_v2_tpiRecon
H093_v2_tpiRecon
H090_v2_tpiRecon
H090_v2_tpiRecon
H090_v2_tpiRecon
H071_v2_tpiRecon
H071_v2_tpiRecon
H071_v2_tpiRecon
H099_v2_tpiRecon
H099_v2_tpiRecon
H099_v2_tpiRecon
H078_v2_tpiRecon
H078_v2_tpiRecon
H078_v2_tpiRecon
H085_v2_tpiRecon
H085_v2_tpiRecon
H085_v2_tpiRecon
H080_v2_tpiRecon
H080_v2_tpiRecon
H080_v2_tpiRecon
H095_v2_tpiRecon
H095_v2_tpiRecon
H095_v2_tpiRecon
Das_H052_v2
Das_H052_v2
Das_H052_v2
Das_H052_v2
H074_v2_tpiRecon
H074_v2_tpiRecon
H074_v2_tpiRecon
H089_v2_tpiRecon
H089_v2_tpiRecon
H089_v2_tpiRecon
H092_v2_tpiRecon
H092_v2_tpiRecon
H092_v2_tpiRecon
H073_v2_tpiRecon
H073_v2_tpiRecon
H0

100%|██████████| 64/64 [01:04<00:00,  1.01s/it]


Components: Auto
PSNR: 30.01 dB
Sigma: 2.79e+01
Components used: 1.1691207885742188

Processing subject 49

Analyzing n_components = None


100%|██████████| 64/64 [01:01<00:00,  1.04it/s]


Components: Auto
PSNR: 28.90 dB
Sigma: 3.25e+01
Components used: 1.5365829467773438

Processing subject 96

Analyzing n_components = None


...

Components: Auto
PSNR: 15.24 dB
Sigma: 7.70e+01
Components used: 0.0

Processing subject 70

Analyzing n_components = None


100%|██████████| 64/64 [01:14<00:00,  1.17s/it]


Components: Auto
PSNR: 29.19 dB
Sigma: 2.70e+01
Components used: 1.077056884765625

Processing subject 91

Analyzing n_components = None


100%|██████████| 64/64 [01:13<00:00,  1.15s/it]


Components: Auto
PSNR: 27.42 dB
Sigma: 2.95e+01
Components used: 1.0499191284179688

Generating aggregate plots and statistics...

Analysis complete! Results saved in: mppca_analysis_results

Summary of results:
  Components  PSNR_mean  PSNR_std   PSNR_min   PSNR_max  Sigma_mean  \
0       None  29.350199  2.935116  15.243155  33.095733    27.97121   

   Sigma_std  NPars_mean  NPars_std  
0   9.566662    1.129679   0.250307  
